In [1]:
import sys
import time

import numpy as np
import scipy.sparse as sp
from scipy.integrate import solve_ivp
from scipy.linalg import expm as expm
import matplotlib.pyplot as plt

sys.path.append('./../lib')
from intertrain import Intertrain
from solver import Solver as Solver

from IPython.core.display import HTML
with open( './style.css', 'r' ) as f: st = f.read()
HTML('<style>%s</style>' %st)

<div class="head1">
    <div class="head1__name">
        Custom solver
    </div>
</div>

In [2]:
def rk4(f, r0, t_min, t_max, t_poi):
    h = (t_max - t_min) / (t_poi - 1)
    t = t_min
    r = r0

    for _ in range(1, t_poi):
        k1 = h * f(r, t)
        k2 = h * f(r + 0.5 * k1, t + 0.5 * h)
        k3 = h * f(r + 0.5 * k2, t + 0.5 * h)
        k4 = h * f(r + k3, t + h)
        t+= h
        r+= (k1 + k2 + k2 + k3 + k3 + k4) / 6.
        
    return r

<div class="head1">
    <div class="head1__name">
        Test 1D
    </div>
</div>

Consider
$$
    d r = f(r, t) \, d t,
    \quad
    r \in R,
    \quad
    r(0) = r_0,
$$
where
$$
    f(r, t) = r^2.
$$

Analytic solution is
$$
    r(t) = \frac{r_0}{1 - r_0 t}.
$$

In [3]:
def func_f0(r, t):
    return r*r

def func_rt(t):
    return r0 / (1. - r0 * t)

In [4]:
t_min = 0.
t_max = 1.
t_poi = 1000
r0 = -5.5 * np.arange(10)

<div class="head2">
    <div class="head2__name">
        Solution by custom solver
    </div>
</div>

In [5]:
_t = time.time()
r_calc = rk4(func_f0, r0, t_min, t_max, t_poi)
_t = time.time() - _t

r_real = func_rt(t_max)

_e = np.mean(np.abs(r_real - r_calc))

print('Error (mean) : %-8.2e'%_e)
print('Time (sec)   : %-8.2e'%_t)

Error (mean) : 4.15e-01
Time (sec)   : 1.85e-02


<div class="head1">
    <div class="head1__name">
        Test 2D
    </div>
</div>

Consider
$$
    d r = f(r, t) \, d t,
    \quad
    r = (r_1, r_2) \in R^{2},
    \quad
    r(0) = r_0,
$$

Let set analytic solution
$$
    r_1(t) = \sin{2 t^2} + r_{0, 1},
    \quad
    r_2(t) = \cos{2 t^2} + r_{0, 2} - 1.
$$
then for the rhs we have
$$
    f_1(r, t) = 4 t \cos{2 t^2} = 4 t (r_2 - r_{0, 2} + 1),
    \quad
    f_2(r, t) = - 4 t \sin{2 t^2} = - 4 t (r_1 - r_{0, 1}).
$$

In [6]:
def func_f0(r, t):
    v1 = +4. * t * (r[1, :] - r0[1, :] + 1.)
    v2 = -4. * t * (r[0, :] - r0[0, :])
    return np.vstack([v1, v2])

def func_f0_el(r, t, r0):
    v1 = +4. * t * (r[1, :] - r0[1, :] + 1.)
    v2 = -4. * t * (r[0, :] - r0[0, :])
    return np.vstack([v1, v2])

def func_rt(t):
    v1 = np.sin(2. * t * t) + r0[0, :]
    v2 = np.cos(2. * t * t) + r0[1, :] - 1.
    return np.vstack([v1, v2])

In [7]:
t_min = 0.
t_max = 0.001
t_poi = 10000
r0 = np.vstack([
    np.arange(10) * 1.1,
    np.arange(10) * 1.5,
])
#r0 = np.array([[2.], [0.]])

<div class="head2">
    <div class="head2__name">
        Solution by custom solver
    </div>
</div>

In [8]:
_t = time.time()
r_calc = rk4(func_f0, r0, t_min, t_max, t_poi)
_t = time.time() - _t

r_real = func_rt(t_max)

_e = np.mean(np.abs(r_real - r_calc))

print('Error (mean) : %-8.2e'%_e)
print('Time (sec)   : %-8.2e'%_t)

Error (mean) : 1.00e-06
Time (sec)   : 5.20e-01


<div class="head2">
    <div class="head2__name">
        Solution by scipy solver
    </div>
</div>

In [9]:
_t = time.time()

r = np.zeros(r0.shape)
for j in range(r0.shape[1]):
    def func(t, r):
        f = func_f0_el(r.reshape(-1, 1), t, r0[:, j].reshape(-1, 1)).reshape(-1)
        return f

    y1 = r0[:, j]
    y2 = solve_ivp(func, [t_min, t_max], y1).y[:, -1]
    r[:, j] = y2
r_calc = r
_t = time.time() - _t

r_real = func_rt(t_max)

_e = np.mean(np.abs((r_real - r_calc) / r_real))

print('Error (mean) : %-8.2e'%_e)
print('Time (sec)   : %-8.2e'%_t)

Error (mean) : 2.22e-06
Time (sec)   : 1.12e-02


<div class="end"></div>